* DWR - Doppler Weather Radar
* Rainfall Rate - per hour
* Dataset used - 10 min data of 1 day only - computation/execution b/c of High End Server issue - (Technical Issue)
* Metric - according to used file - dataset increase - accuracy - modify paramaeter (Technical Issue)
* Model Prep - clean, test/train 70:30, hyperparameter set, model fit
* Class 0 - No Rain
* Class 1 - Rain

# Classification of Radar Rainfall Rate using Machine Learning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ***IMPORTING LIBRARIES***

In [ ]:
!pip install netCDF4
!pip install h5netcdf
!pip install plotly
!pip install pyscancf
!pip install arm-pyart
!pip install cartopy
!pip install pyproj
!pip install wradlib
!pip install xarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 99.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully u

In [ ]:
import netCDF4 as nc
import h5netcdf
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from google.colab import files
from scipy.spatial import cKDTree
import io
import math
import xarray as xr
import struct
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from shapely.geometry import Point
import geopandas as gpd
from pyscancf import pyscancf as pcf
from sklearn.ensemble import GradientBoostingRegressor
import pyart
import pyproj
import wradlib as wrl
from pyart.core import antenna_to_cartesian
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## Cite PyScanCf:

## Syed, H. A.,Sayyed, I., Kalapureddy, M. C. R., & Grandhi, K. K.(2021). 
## PyScanCf – The library for individual sweep datasets of IMD weather radars. 
## Zenodo. doi:10.5281/zenodo.5574160.



# ***Variables***

# Merge Files

In [ ]:
#inp = "/content/drive/MyDrive/imd_radar/DLI200813IMD-B"
#out = "/content/drive/MyDrive/1day_out"

In [ ]:
#pcf.cfrad(input_dir=inp, output_dir=out, scan_type="B", gridder=True)

In [ ]:
radar1=  pyart.io.read("/content/drive/MyDrive/1day_out/cfrad_DLI200813050229-IMD-B.nc")
radar2 = pyart.io.read("/content/drive/MyDrive/1day_out/cfrad_DLI200813045229-IMD-B.nc")
radar3=  pyart.io.read("/content/drive/MyDrive/1day_out/cfrad_DLI200813043229-IMD-B.nc")

In [ ]:
def display_graph(radar):
    display = pyart.graph.RadarDisplay(radar)

    # Set the number of sweeps and the layout (3 columns)
    num_sweeps = 10
    ncols = 3
    nrows = (num_sweeps + ncols - 1) // ncols  # Compute number of rows needed

    # Create a figure and axes for subplots
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows * 5))

    # Flatten the axes array for easy indexing
    axes = axes.flatten()

    # Plot each sweep
    for sweep in range(num_sweeps):
        ax = axes[sweep]
        display.plot('REF', sweep=sweep, ax=ax)
        ax.set_title(f'Sweep {sweep}')
        ax.set_xlim(-50, 50)  # x-axis limit in km
        ax.set_ylim(-50, 50)  # y-axis limit in km

        # Add label
        label_text = 'Radar Location'
        location = (28.58975, 77.22195)
        display.plot_label(label_text, location, ax=ax)

    # Turn off any unused subplots
    for i in range(num_sweeps, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

# Assuming 'radar' is already defined
display_graph(radar2)


# Basic Info of merged file

In [ ]:
radar2.info()

altitude:
	data: <ndarray of type: float32 and shape: (1,)>
	long_name: Altitude
	units: meters
	standard_name: Altitude
	positive: up
altitude_agl: None
antenna_transition:
	data: <ndarray of type: float64 and shape: (3600,)>
azimuth:
	data: <ndarray of type: float32 and shape: (3600,)>
	long_name: azimuth_angle_from_true_north
	units: degrees
	standard_name: beam_azimuth_angle
	axis: radial_azimuth_coordinate
	comment: Azimuth of antenna relative to true north
elevation:
	data: <ndarray of type: float32 and shape: (3600,)>
	long_name: elevation_angle_from_horizontal_plane
	units: degrees
	standard_name: beam_elevation_angle
	axis: radial_elevation_coordinate
	comment: Elevation of antenna relative to the horizontal plane
fields:
	REF:
		data: <ndarray of type: float32 and shape: (3600, 831)>
		long_name: Reflectivity
		units: dBZ
		standard_name: equivalent_reflectivity_factor
		coordinates: elevation azimuth range
	VEL:
		data: <ndarray of type: float32 and shape: (3600, 831)>
		lon

In [ ]:
variables = ['Z', 'V', 'W']
field_names = {
    'Z': 'REF',         # Reflectivity
    'V': 'VEL',         # Velocity
    'W': 'WIDTH',       # Spectrum Width
}

# Initialize a dictionary to store flattened data
data = {var: [] for var in variables}

# Extract and flatten data for each field
for field, radar_field in field_names.items():
    if radar_field in radar2.fields:
        field_data = radar2.fields[radar_field]['data']

        if field_data.ndim == 3:  # (sweeps, rays, gates)
            field_data = field_data.flatten()
        elif field_data.ndim == 2:  # (rays, gates)
            field_data = field_data.flatten()

        # Store non-masked values
        data[field] = field_data[~np.ma.getmaskarray(field_data)]
    else:
        print(f"{radar_field} field is not available in radar data.")

# Plot histograms
plt.figure(figsize=(15, 10))
for i, var in enumerate(variables):
    plt.subplot(3, 3, i+1)

    if var in data and len(data[var]) > 0:
        sns.histplot(data[var], kde=True, color='blue', bins=30)
        plt.title(f'Histogram of {var}')
        plt.xlabel(var)
        plt.ylabel('Frequency')
    else:
        plt.title(f'{var} data not available')
        plt.xlabel(var)
        plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
ref_array = radar2.fields['REF']['data']
lat_array = radar2.gate_latitude['data']
lon_array = radar2.gate_longitude['data']
sweep_number = radar2.sweep_number['data']

# Composite Reflectivity

In [ ]:
def find_x_y_displacement(radar, longitude, latitude):
    """ Return the x and y displacement (in meters) from a radar location. """
    lat_0 = radar.latitude['data'][0]
    lon_0 = radar.longitude['data'][0]
    proj = pyproj.Proj(proj='aeqd', lon_0=lon_0, lat_0=lat_0)  # Azimuthal Equidistant Projection
    x_disp, y_disp = proj(longitude, latitude)
    return x_disp, y_disp

def find_nearest_gate(radar, longitude, latitude, altitude):
    """ Return the indices of the nearest gate to a given point. """
    gate_x = radar.gate_x['data']
    gate_y = radar.gate_y['data']
    gate_z = radar.gate_z['data']

    x_disp, y_disp = find_x_y_displacement(radar, longitude, latitude)
    distances = np.sqrt(
        (gate_x - x_disp)**2. + (gate_y - y_disp)**2. + (gate_z - altitude)**2.)
    return np.unravel_index(distances.argmin(), distances.shape)

def get_reflectivity_at_location(radar, longitude, latitude, altitude):
    """ Retrieve reflectivity at the nearest gate to the given location. """
    ray, gate = find_nearest_gate(radar, longitude, latitude, altitude)

    # Retrieve reflectivity data at the nearest gate
    gate_latitude = radar.gate_latitude['data'][ray, gate]
    gate_longitude = radar.gate_longitude['data'][ray, gate]
    gate_altitude = radar.gate_altitude['data'][ray, gate]
    gate_reflectivity = radar.fields['REF']['data'][ray, gate]

    return {
        "Latitude": gate_latitude,
        "Longitude": gate_longitude,
        "Altitude": gate_altitude,
        "Reflectivity": gate_reflectivity
    }

# Define the locations to check
locations = {
    "New Delhi HQ": [28.589, 77.222],
    "Badli": [28.745, 77.138],
    "Coronation Pilla": [28.726, 77.187],
    "Gurmandi": [28.686, 77.197],
    "Kanjhawala": [28.738, 77.008],
    "Lodi Road": [28.589, 77.225],
    "Mahrauli": [28.520, 77.178],
    "Nangloi": [28.682, 77.068],
    "New Delhi OBSY": [28.614, 77.209]
}

# List to store results


# Process each location
def ref(radar):
  results = []
  for name, (lat, lon) in locations.items():
    result = get_reflectivity_at_location(radar, lon, lat, altitude=253)
    result["Location Name"] = name
    results.append(result)
  return results
results1=ref(radar1)
ressults2=ref(radar2)
results3=ref(radar3)
Timelist=['05:02:29','04:52:29','04:32:29']
# Convert results to a DataFrame
df1 = pd.DataFrame(results1)
df2 = pd.DataFrame(ressults2)
df3 = pd.DataFrame(results3)
df1['Time'] = pd.to_datetime(Timelist[0], format='%H:%M:%S').time()
df2['Time'] = pd.to_datetime(Timelist[1], format='%H:%M:%S').time()
df3['Time'] = pd.to_datetime(Timelist[1], format='%H:%M:%S').time()
df1=df1[["Location Name", "Time","Latitude", "Longitude", "Reflectivity"]]
df2=df2[["Location Name", "Time","Latitude", "Longitude", "Reflectivity"]]
df3=df3[["Location Name", "Time","Latitude", "Longitude", "Reflectivity"]]
df = pd.concat([df1, df2, df3], ignore_index=True)

# Display the combined DataFrame
print(df)

# ***Extracting Rainfall from Reflectivity***

In [ ]:
dBZ = np.array(df['Reflectivity'] )
print(dBZ)

In [ ]:
#converting Z to linear
Z = wrl.trafo.idecibel(dBZ)
print(Z)

In [ ]:
#Linear_Z to rain rate
R = wrl.zr.z_to_r(Z, a=200.0, b=1.6)
print(np.round(R, 2))

In [ ]:
depth = wrl.trafo.r_to_depth(R, 600)
print(np.round(depth, 2))

In [ ]:
df['Rain_Rate']=np.round(R, 2)
df

# ***Classification***

In [ ]:
df.isnull().sum()

In [ ]:
rain_df=df.copy()

In [ ]:
rain_df['Rain_class'] = rain_df['Rain_Rate'].apply(lambda x: 1 if x > 0.99 else 0)
rain_df

In [ ]:
x_data=rain_df.iloc[:,2:-1]
y_data=rain_df['Rain_class']
x_data

In [ ]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=4, random_state=42),
    'SVM': SVC(kernel='linear', C=1, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=3),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

In [ ]:
def evaluate(x_train,y_train,y_test,x_test,models):
  for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.2f}")
    print(f"\n{name} Classification Report:\n")
    print(classification_report(y_test, y_pred))

In [ ]:
evaluate(x_train,y_train,y_test,x_test,models)

In [ ]:
def precision_model(x_train,y_train,x_test,y_test,models):
  precisions = {}
  # Train, predict, and calculate classification report for each model
  for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # Get classification report as dictionary
    report = classification_report(y_test, y_pred, output_dict=True)

    # Extract precision for both classes (0 and 1)
    precisions[name] = {
        'Class 0': report['0']['precision'],
        'Class 1': report['1']['precision']
    }

  # Convert precisions dictionary to DataFrame for easier plotting
  precisions_df = pd.DataFrame(precisions)

  # Plot the precisions using a bar graph
  precisions_df.T.plot(kind='bar', figsize=(10, 6))
  plt.title('Model Precisions for Class 0 and Class 1')
  plt.ylabel('Precision')
  plt.xlabel('Models')
  plt.xticks(rotation=45)
  plt.legend(title="Classes")
  plt.tight_layout()
  plt.show()

In [ ]:
precision_model(x_train,y_train,x_test,y_test,models)